In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/1.year.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            data = {'cssClass': 'form-control',
                    'dropdownType': '1',
                    'selectedValue': '',
                    'vehicleAttributes': {'Make': '',
                                          'MakeID': 0,
                                          'ModelID': 0,
                                          'SkinID': '1',
                                          'Year': f'''{input_.loc[a, 'Year']}'''}}
            
            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.post('https://www.dormanproducts.com/ApplicationGuideService.asmx/BuildDropdown',
                                         data=json.dumps(data),
                                         headers=get_header(**{'Content-Type': 'application/json; charset=UTF-8'}),
                                         proxies=get_proxy(),
                                         timeout=(10, 10))

                    if resp.status_code == 200:
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            json_ = resp.json()

            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(json_['d'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_make_code = html.xpath('//select[@id="makeDropdown"]/option/@value')[1:]

            # = = = = = = = = = = = = = = =

            list_make = html.xpath('//select[@id="makeDropdown"]/option/text()')[1:]

            # = = = = = = = = = = = = = = =

            if not list_make:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Make': list_make,
                                    'Make Code': list_make_code,
                                    'Year': input_.loc[a, 'Year']})
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Year']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make', 'Year'],
                                                ascending=[True, False],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/2.make-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Year'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/make_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：102

[状态：ok，尝试次数：1] - 2013
[剩余数量：72] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2003
[剩余数量：71] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2004
[剩余数量：70] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2019
[剩余数量：69] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2016
[剩余数量：68] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 1999
[剩余数量：67] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 1997
[剩余数量：66] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2008
[剩余数量：65] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2002
[剩余数量：64] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2018
[剩余数量：63] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2024
[剩余数量：62] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2005
[剩余数量：61] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2011
[剩余数量：60] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2007
[剩余数量：59] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 1996
[剩余数量：58] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2014
[剩余数量：57] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2025
[剩余数量：56] - [当前时间：18:49:19]

[状态：ok，尝试次数：1] - 2020
[剩余数量：55] - [当前时间：18:49:20]

[状态：ok，尝试次数：1] - 2001
[剩余数量：54] - [当前时间：18:49:20]

[状态：ok，尝试次数：1] - 2006
